<a href="https://colab.research.google.com/github/Shashank1202/Kaggle_playground/blob/main/kaggle20_Restaurant_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
import pandas as pd

file_path='Restaurant_Reviews.tsv'
data=pd.read_csv(file_path, sep='\t')

In [5]:
data.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [6]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
data['Liked'].value_counts()

1    500
0    500
Name: Liked, dtype: int64

In [8]:
data.isnull().sum()

Review    0
Liked     0
dtype: int64

In [9]:
data.shape

(1000, 2)

In [10]:
for i in range(5):
  print("Review ",[i])
  print(data['Review'].iloc[i],"\n")
  print("Review",data['Liked'].iloc[i],"\n\n")

Review  [0]
Wow... Loved this place. 

Review 1 


Review  [1]
Crust is not good. 

Review 0 


Review  [2]
Not tasty and the texture was just nasty. 

Review 0 


Review  [3]
Stopped by during the late May bank holiday off Rick Steve recommendation and loved it. 

Review 1 


Review  [4]
The selection on the menu was great and so were the prices. 

Review 1 




In [11]:
def no_of_words(text):
  words=text.split()
  word_count=len(words)
  return word_count

In [12]:
data['word count']=data['Review'].apply(no_of_words)

In [13]:
data.head()

,Review,Liked,word count
0,Wow... Loved this place.,1,4
1,Crust is not good.,0,4
2,Not tasty and the texture was just nasty.,0,8
3,Stopped by during the late May bank holiday of...,1,15
4,The selection on the menu was great and so wer...,1,12


In [14]:
from matplotlib import style
style.use('ggplot')
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
def data_processing(text):
  text=text.lower()
  text=re.sub('<br />', '', text)
  text=re.sub(r"https\S+|www\S+|http\S+", '', text, flags=re.MULTILINE)
  text=re.sub(r'[^\w\s]', '', text)
  text_tokens=word_tokenize(text)
  filtered_text=[w for w in text_tokens if not w in stop_words]
  return " ".join(filtered_text)

In [16]:
data.Review=data['Review'].apply(data_processing)

In [17]:
duplicated_counts=data.duplicated().sum()
print("Number of duplicated entries :",duplicated_counts)

Number of duplicated entries : 5


In [18]:
data=data.drop_duplicates('Review')

In [19]:
data.shape

(990, 3)

In [20]:
stemmer=PorterStemmer()
def stemming(data):
  text=[stemmer.stem(word) for word in data]
  return data

In [21]:
data.Review=data['Review'].apply(lambda x: stemming(x))

In [22]:
like=data[data.Liked==1]
like.head()

,Review,Liked,word count
0,wow loved place,1,4
3,stopped late may bank holiday rick steve recom...,1,15
4,selection menu great prices,1,12
8,fries great,1,5
9,great touch,1,3


In [23]:
from collections import Counter
count=Counter()
for text in like['Review'].values:
  for word in text.split():
    count[word]+=1
count.most_common(15)

[('good', 72),
 ('great', 70),
 ('food', 59),
 ('place', 56),
 ('service', 46),
 ('friendly', 23),
 ('back', 23),
 ('nice', 22),
 ('delicious', 22),
 ('time', 22),
 ('really', 22),
 ('best', 22),
 ('amazing', 21),
 ('also', 18),
 ('like', 17)]

In [24]:
dislike=data[data.Liked==0]
dislike.head()

,Review,Liked,word count
1,crust good,0,4
2,tasty texture nasty,0,8
5,getting angry want damn pho,0,11
6,honeslty didnt taste fresh,0,6
7,potatoes like rubber could tell made ahead tim...,0,22


In [25]:
for text in dislike['Review']:
  for word in text.split():
    count[word]+=1
count.most_common(15)

[('food', 122),
 ('place', 104),
 ('good', 94),
 ('service', 83),
 ('great', 70),
 ('back', 58),
 ('like', 46),
 ('go', 43),
 ('time', 42),
 ('really', 36),
 ('best', 30),
 ('ever', 28),
 ('dont', 28),
 ('also', 27),
 ('would', 27)]

In [26]:
x=data['Review']
y=data['Liked']

In [27]:
vect=TfidfVectorizer()
x=vect.fit_transform(data['Review'])

In [28]:
x_train,x_test,y_train,y_test=train_test_split(x, y, test_size=0.2, random_state=42)

In [29]:
print("x train size: ",x_train.shape)
print("x test size: ", x_test.shape)
print("y train size: ",y_train.shape)
print("y test size: ",y_test.shape)

x train size:  (792, 1956)
x test size:  (198, 1956)
y train size:  (792,)
y test size:  (198,)


In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [31]:
log=LogisticRegression()
log.fit(x_train,y_train)
log_pred=log.predict(x_test)
log_acc=accuracy_score(log_pred,y_test)
print("Test accuaracy :{:.2f}%".format(log_acc*100))

Test accuaracy :75.76%


In [33]:
print(confusion_matrix(y_test,log_pred))
print("\n")
print(classification_report(y_test,log_pred))

[[85 14]
 [34 65]]


              precision    recall  f1-score   support

           0       0.71      0.86      0.78        99
           1       0.82      0.66      0.73        99

    accuracy                           0.76       198
   macro avg       0.77      0.76      0.76       198
weighted avg       0.77      0.76      0.76       198



In [38]:
mnb=MultinomialNB()
mnb.fit(x_train,y_train)
mnb_pred=mnb.predict(x_test)
mnb_acc=accuracy_score(mnb_pred, y_test)
print("Test accuracy: {:.2f}%".format(mnb_acc*100))

Test accuracy: 77.27%


In [39]:
print(confusion_matrix(mnb_pred, y_test))
print("\n")
print(classification_report(mnb_pred,y_test))

[[75 21]
 [24 78]]


              precision    recall  f1-score   support

           0       0.76      0.78      0.77        96
           1       0.79      0.76      0.78       102

    accuracy                           0.77       198
   macro avg       0.77      0.77      0.77       198
weighted avg       0.77      0.77      0.77       198



In [42]:
svc=LinearSVC()
svc.fit(x_train,y_train)
svc_pred=svc.predict(x_test)
svc_acc=accuracy_score(svc_pred, y_test)
print("Test accuracy: {:.2f}%".format(svc_acc*100))

Test accuracy: 77.78%


In [43]:
print(confusion_matrix(mnb_pred, y_test))
print("\n")
print(classification_report(mnb_pred,y_test))

[[75 21]
 [24 78]]


              precision    recall  f1-score   support

           0       0.76      0.78      0.77        96
           1       0.79      0.76      0.78       102

    accuracy                           0.77       198
   macro avg       0.77      0.77      0.77       198
weighted avg       0.77      0.77      0.77       198



In [46]:
from sklearn.ensemble import RandomForestClassifier

rf=RandomForestClassifier()
rf.fit(x_train,y_train)
rf_pred=rf.predict(x_test)
rf_acc=accuracy_score(rf_pred, y_test)
print("Test accuracy: {:.2f}%".format(rf_acc*100))

Test accuracy: 75.76%


In [48]:
!pip install hmmlearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 4.8 MB/s eta 0:00:00


In [54]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset, random_split
from tqdm import tqdm

In [55]:
data=data[['Review', 'Liked']]

In [56]:
train_data, test_data=train_test_split(data, test_size=0.2, random_state=42)

In [59]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#tokenize and encode the review texts
train_encodings = tokenizer(list(train_data['Review']), truncation=True, padding=True, max_length=256, return_tensors='pt')
test_encodings= tokenizer(list(test_data['Review']), truncation=True, padding= True, max_length=256, return_tensors='pt')

#convert labels to pytorch tensors
train_labels= torch.tensor(list(train_data['Liked']))
test_labels= torch.tensor(list(test_data['Liked']))

In [60]:
# Create DataLoader for training and testing sets
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [61]:
# Load pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [62]:
# Set up optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

# Training loop
epochs = 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(epochs):
    model.train()
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}'):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Optional: Evaluate on the test set after each epoch
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for batch in test_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=1)

            total += labels.size(0)
            correct += (predictions == labels).sum().item()

        accuracy = correct / total
        print(f'Epoch {epoch + 1}, Accuracy on Test Set: {accuracy:.4f}')


Epoch 1: 100%|██████████| 99/99 [03:07<00:00,  1.90s/it]


Epoch 1, Accuracy on Test Set: 0.6515


Epoch 2: 100%|██████████| 99/99 [02:47<00:00,  1.70s/it]


Epoch 2, Accuracy on Test Set: 0.7980


Epoch 3: 100%|██████████| 99/99 [02:48<00:00,  1.70s/it]


Epoch 3, Accuracy on Test Set: 0.8182


In [65]:
# Set model to evaluation mode
model.eval()

# Predict on a new review
new_review = "This restaurant has great food!"
input_encoding = tokenizer(new_review, truncation=True, padding=True, max_length=256, return_tensors='pt')

with torch.no_grad():
    output = model(**input_encoding)
    probabilities = torch.softmax(output.logits, dim=1)

# Interpret the result
predicted_label = torch.argmax(probabilities).item()
print(f'Predicted Label: {predicted_label}, Probabilities: {probabilities}')


Predicted Label: 1, Probabilities: tensor([[0.0076, 0.9924]])


In [66]:
# Set model to evaluation mode
model.eval()

# Predict on a new review
new_review = "food was not good"
input_encoding = tokenizer(new_review, truncation=True, padding=True, max_length=256, return_tensors='pt')

with torch.no_grad():
    output = model(**input_encoding)
    probabilities = torch.softmax(output.logits, dim=1)

# Interpret the result
predicted_label = torch.argmax(probabilities).item()
print(f'Predicted Label: {predicted_label}, Probabilities: {probabilities}')

Predicted Label: 0, Probabilities: tensor([[0.9864, 0.0136]])
